# Notebook MikuSec

## Setup Google Colab (not necessary if running in local environment)

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = "/content/drive/Shareddrives/MikuSec/"

Mounted at /content/drive


## Setup Telegram API

https://www.codementor.io/@garethdwyer/building-a-telegram-bot-using-python-part-1-goi5fncay

In [2]:
import json
import requests
import time
import urllib

token = "" #Insert here your Chatbot token

## Setup OwlReady2 Library

https://owlready2.readthedocs.io/en/v0.35/intro.html#short-example-what-can-i-do-with-owlready

In [3]:
!pip3 install owlready2

import owlready2

from owlready2 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 24.8 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.38-cp37-cp37m-linux_x86_64.whl size=22127077 sha256=984003a8955b7c157d9ff5dc35134f1965aee1583096f275941070cdbdaf7beb
  Stored in directory: /root/.cache/pip/wheels/ee/48/58/3ea20d69efcb97a26048fbfcce9332412077df80d4ae52365b
Successfully built owlready2


In [4]:
onto_path.append(path)
onto = get_ontology("mikusec-tsundere.owl") # mikusec-padrao.owl OR mikusec-agressividade.owl OR mikusec-tsundere.owl
onto.load()

get_ontology("http://www.semanticweb.org/mikusec-tsundere#")

In [5]:
onto.base_iri

'http://www.semanticweb.org/mikusec-tsundere#'

In [6]:
list(onto.classes())

[mikusec-tsundere.pergunta, mikusec-tsundere.resposta]

In [7]:
list(onto.individuals())

[mikusec-tsundere.Q0,
 mikusec-tsundere.Q1,
 mikusec-tsundere.Q2,
 mikusec-tsundere.Q3,
 mikusec-tsundere.R0,
 mikusec-tsundere.R1,
 mikusec-tsundere.R2,
 mikusec-tsundere.R3,
 mikusec-tsundere.R4]

In [8]:
onto.Q1.comment[0]

'Existe uma entidade confiável no sistema? (responda sim ou não)'

In [9]:
onto.R0.comment[0]

'CAACAgQAAxkBAAEFU6li2GQYsK1wXNq8nf4GGA4RZW-rtwACDQMAAtkjZCF0EdiV1BOCMSkE'

In [10]:
onto.R0.comment[1]

'Desculpe, não entendi...'

## Loop

In [11]:
TOKEN = token
URL = "https://api.telegram.org/bot{}/".format(TOKEN)

estado = {}

def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates(offset=None):
    url = URL + "getUpdates"
    if offset:
        url += "?offset={}".format(offset)
    js = get_json_from_url(url)
    return js


def get_last_update_id(updates):
    update_ids = []
    for update in updates["result"]:
        update_ids.append(int(update["update_id"]))
    return max(update_ids)


def echo_all(updates):
    for update in updates["result"]:
        chat = update["message"]["chat"]["id"]
        text = update["message"]["text"]
        print(chat, text)

        try:

          if ((text == onto.Q0.comment[0]) and (estado[chat] == 0)):
            question = onto.Q1.comment[0]
            send_message(question, chat)

            estado[chat] = 1
            print(estado)

            return

          if ((text == "sim") and (estado[chat] == 1)):
            response = onto.R1.comment[1]
            send_message(response, chat)
            send_sticker(onto.R1.comment[0], chat)

            estado[chat] = 0
            print(estado)

            return

          if ((text == "não") and (estado[chat] == 1)):
            question = onto.Q2.comment[0]
            send_message(question, chat)

            estado[chat] = 2
            print(estado)

            return

          if ((text == "não") and (estado[chat] == 2)):
            response = onto.R2.comment[1]
            send_message(response, chat)
            send_sticker(onto.R2.comment[0], chat)

            estado[chat] = 0
            print(estado)

            return

          if ((text == "sim") and (estado[chat] == 2)):
            question = onto.Q3.comment[0]
            send_message(question, chat)

            estado[chat] = 3
            print(estado)

            return

          if ((text == "sim") and (estado[chat] == 3)):
            response = onto.R3.comment[1]
            send_message(response, chat)
            send_sticker(onto.R3.comment[0], chat)

            estado[chat] = 0
            print(estado)

            return

          if ((text == "não") and (estado[chat] == 3)):
            response = onto.R4.comment[1]
            send_message(response, chat)
            send_sticker(onto.R4.comment[0], chat)

            estado[chat] = 0
            print(estado)

            return

          else:
            send_message(onto.R0.comment[1], chat)
            send_sticker(onto.R0.comment[0], chat)

            estado[chat] = 0
            print(estado)

            return

        except:
          estado[chat] = 0


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (text, chat_id)


def send_message(text, chat_id):
    text = urllib.parse.quote_plus(text)
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)


def send_sticker(sticker_id, chat_id):
    url = URL + "sendSticker?chat_id={}&sticker={}".format(chat_id, sticker_id)
    get_url(url)    


def main():
    last_update_id = None
    while True:
        updates = get_updates(last_update_id)
        if len(updates["result"]) > 0:
            last_update_id = get_last_update_id(updates) + 1
            echo_all(updates)

            for update in updates["result"]:
              chat = update["message"]["chat"]["id"]

        time.sleep(0.5)


if __name__ == '__main__':
    main()

986850750 /start
{986850750: 0}
1910884100 Oi
{986850750: 0, 1910884100: 0}
1910884100 onegai
{986850750: 0, 1910884100: 1}
1910884100 sim
{986850750: 0, 1910884100: 0}


KeyboardInterrupt: ignored